## Set-up and data loading

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from lib import data

sns.set()

%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [2]:
data.download_data()

The data is already downloaded.


In [3]:
header, dtype_dict = data.get_data_header_and_dtypes()
learn_df = pd.read_csv(
    "data/us_census_full/census_income_learn.csv",
    names=header,
    usecols=[i for i in range(42) if i != 24],
    dtype=dtype_dict,
)
test_df = pd.read_csv(
    "data/us_census_full/census_income_test.csv",
    names=header,
    usecols=[i for i in range(42) if i != 24],
    dtype=dtype_dict,
)

## Exploratory data analysis